
<p style="padding: 30px;
          color: #E67E22;
          text-align: center;
          font-family:Verdana;
          font-size:330%;">
👀 Amazon - Review Sentiment Analysis 👀 🐳 🧐 

</p>

<img algin="center" src="https://github.com/TarikKaanKoc/Rating.Product-and-Sorting.Reviews-in-Amazon/blob/main/image.jpeg?raw=True"/>

<div class="alert alert-primary" style="margin-top: 20px">

<strong><h2>Business Problem:</h2></strong>
<blockquote><p>One of the most important problems in e-commerce is the correct calculation of points.
given to aftermarket products. Solving this problem means providing more.
Customer satisfaction for e-commerce site, product prominence for sellers
and a seamless shopping experience for buyers. Another problem is correct ordering.
The prominence of misleading comments among the comments made on the products
financial loss and loss of customers. <strong>That's why amazon company wants to rank reviews, and when ordering these reviews, they want to rank them according to their sentiment (positive / negative).</strong> In solving these 2 basic problems,
e-commerce site and sellers will increase their sales, customers
hassle-free purchasing journey.</p>

<strong><h2>Dataset Story:</h2></strong>
<blockquote><p>
This dataset containing Amazon Product Data includes product categories and various metadata.
The product with the most comments in the electronics category has user ratings and comments. </p>
</blockquote>
<ul>
 

</ul>
</div>

----
---

In [ ]:
!pip install SentimentIntensityAnalyzer
!pip install chart_studio
!pip install TextBlob
!pip install plotly
!pip install WordCloud
!pip install cufflinks

In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import nltk
import re
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline();
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option('display.max_columns', None)

<p style="padding: 30px;
          color: #E67E22;
          text-align: center;
          font-family:Verdana;
          font-size:330%;">
👀 Reading the Dataset and an Overview 👀 🧐 

</p>

In [ ]:
df_ = pd.read_csv("../input/amazon/amazon_reviews.csv")

In [ ]:
df = df_.copy()

In [ ]:
df = df.sort_values("wilson_lower_bound", ascending=False)
df.drop('Unnamed: 0', inplace = True, axis = 1)
df.head()

In [ ]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

def check_dataframe(df, head=5, tail = 5):
    
    print(" SHAPE ".center(82,'~'))
    print('Rows: {}'.format(df.shape[0]))
    print('Columns: {}'.format(df.shape[1]))
    print(" TYPES ".center(82,'~'))
    print(df.dtypes)
    print("".center(82,'~'))
    print(missing_values_analysis(df))
    print(' DUPLICATED VALUES '.center(83,'~'))
    print(df.duplicated().sum())
    print(" QUANTILES ".center(82,'~'))
    print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_dataframe(df)

In [ ]:
def check_class(dataframe):
    nunique_df = pd.DataFrame({'Variable': dataframe.columns,
                               'Classes': [dataframe[i].nunique() \
                                           for i in dataframe.columns]})

    nunique_df = nunique_df.sort_values('Classes', ascending=False)
    nunique_df = nunique_df.reset_index(drop = True)
    return nunique_df

check_class(df)

In [ ]:
# categorical variable analysis ---> overall

constraints = ['#581845','#C70039','#2E4053','#1ABC9C','#7F8C8D']

def categorical_variable_summary(df, column_name):
    fig = make_subplots(rows=1,cols=2,
                        subplot_titles=('Countplot','Percentages'),
                        specs=[[{"type": "xy"}, {'type':'domain'}]])

    fig.add_trace(go.Bar( y = df[column_name].value_counts().values.tolist(), 
                          x = [str(i) for i in df[column_name].value_counts().index], 
                          text = df[column_name].value_counts().values.tolist(),
                          textfont = dict(size=15),
                          name = column_name,
                          textposition = 'auto',
                          showlegend=False,
                          marker=dict(color = constraints,
                                      line=dict(color='#DBE6EC',
                                                width=1))),
                  row = 1, col = 1)
    
    fig.add_trace(go.Pie(labels= df[column_name].value_counts().keys(),
                         values= df[column_name].value_counts().values,
                         textfont = dict(size = 20),
                         textposition='auto',
                         showlegend = False,
                         name = column_name,
                         marker=dict(colors=constraints)),
                  row = 1, col = 2)
    
    fig.update_layout(title={'text': column_name,
                             'y':0.9,
                             'x':0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')
    
    iplot(fig)

In [ ]:
categorical_variable_summary(df,'overall')

## Everything is going well...
----

<div class="alert alert-block alert-info">
        <b> 📌 Our goal is to rank the comments by sentiment analysis so we don't get hung up on other details. I have sorted this dataset according to certain rules before, if you want to review it :</b>
</div>
<br>

#### 📌 [To access the sorting project please click](https://www.kaggle.com/code/tarkkaanko/rating-product-sorting-reviews-in-amazon)

In [ ]:
# sample for cleaning
df.reviewText.head()

In [ ]:
example_review = df.reviewText[2031]
example_review

In [ ]:
# we clean it from punctuation and numbers - using regex. {Regular expression}
example_review = re.sub("[^a-zA-Z]",' ',example_review)
example_review

## An important point! 

<div class="alert alert-block alert-info">
        <b> 📌I will now convert the text to all lowercase.
Our machine learning algorithms recognize words that start with a capital letter as 
different words, and we will convert them to lowercase. Thus, our machine learning algorithms will not 
perceive words that start with a capital letter as a different word.</b>
</div>
<br>




In [ ]:
example_review = example_review.lower().split()

In [ ]:
example_review

In [ ]:
rt = lambda x: re.sub("[^a-zA-Z]",' ',str(x))
df["reviewText"] = df["reviewText"].map(rt)
df["reviewText"] = df["reviewText"].str.lower()
df.head(10)

In [ ]:
'''
# Sentiment analysis
# TextBlob Exit will return polarity and subjectivity.
# Polarity indicates your mood, that is, whether it is positive.
# It returns a value between 0 and 1. The closer to 1 the more positive, the closer to 0 the more negative.
'''

df[['polarity', 'subjectivity']] = df['reviewText'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

for index, row in df['reviewText'].iteritems():

    score = SentimentIntensityAnalyzer().polarity_scores(row)

    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    if neg > pos:
        df.loc[index, 'sentiment'] = "Negative"
    elif pos > neg:
        df.loc[index, 'sentiment'] = "Positive"
    else:
        df.loc[index, 'sentiment'] = "neutral"

In [ ]:
# 20 Identifying the interpretation, now we can include the positive, negative and neutral status of the comments.

df[df["sentiment"] == "Positive"].sort_values("wilson_lower_bound", ascending=False).head(5)

In [ ]:
# Let's see if we have an unbalanced data problem
categorical_variable_summary(df,'sentiment')

In [ ]:
# Let's see if there is an imbalance in the scoring?
df.groupby(["sentiment"])[['overall']].mean()